In [1]:
!pip install --force-reinstall ../dist/alquimia_fair_forge-0.0.1.tar.gz -q


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [2]:
from fair_forge.metrics import BestOf
from pydantic import SecretStr
from helpers.retriever import LocalRetriever

/Users/alexfiorenza/Documents/software_development/projects/alquimia/fair-forge/venv/lib/python3.13/site-packages/fair_forge/metrics/humanity.py:10: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
/Users/alexfiorenza/Documents/software_development/projects/alquimia/fair-forge/venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from getpass import getpass
judge_api_key = SecretStr(getpass("Please enter your Judge API key: "))

In [5]:
metrics = BestOf.run(
    LocalRetriever,
    judge_api_key=judge_api_key,
    verbose=True
)

2025-10-03 17:30:58,593 - fair_forge - INFO - Loaded dataset with 2 batches
2025-10-03 17:30:58,593 - fair_forge - INFO - [BestOf] Aggregating datasets by query for best-of comparisons
2025-10-03 17:30:58,596 - fair_forge - DEBUG - Comparing 123 and 456 contestants
2025-10-03 17:30:58,596 - fair_forge - DEBUG - Left contestant: 

Query: What is Alquimia AI?
Answer: I'm so happy to answer your question. Alquimia AI Is an startup dedicated to construct assistants.
Expected answer: Is an startup that its aim is to construct assistants


2025-10-03 17:30:58,597 - fair_forge - DEBUG - Right contestant: 

Query: What is Alquimia AI?
Answer: I'm so happy to answer your question. Alquimia AI Is an startup dedicated to construct assistants.
Expected answer: Is an startup that its aim is to construct assistants


Query: What is Alquimia AIssss?
Answer: I'm so happy to answer your question. Alquimia AI Is an sssstartup dedicated to construct assistants.
Expected answer: Is an startup that its aim